# Learn and Predict CSAT using ML algorithms

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'png' #set 'png' here when working on notebook
%matplotlib inline
from scipy import stats

# hide all warnings
import warnings
warnings.filterwarnings('ignore');

In [2]:
train_df = pd.read_csv("input/2018_csat.csv")
test_df = pd.read_csv("input/2019_csat.csv")

In [3]:
y_train_df = train_df.AverageCSAT
train_df.drop('CaseNumber', axis=1, inplace=True)
train_df.drop('DateOpened', axis=1, inplace=True)
train_df.drop('DateInitialResponse', axis=1, inplace=True)
train_df.drop('DateEscalated', axis=1, inplace=True)
y_test_df = test_df.AverageCSAT
test_df.drop('AverageCSAT', axis=1, inplace=True)
test_df.drop('CaseNumber', axis=1, inplace=True)
test_df.drop('DateOpened', axis=1, inplace=True)
test_df.drop('DateInitialResponse', axis=1, inplace=True)
test_df.drop('DateEscalated', axis=1, inplace=True)

In [4]:
# perform one-hot encoding
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)

In [5]:
# remove AverageCSAT from training set after heatmap
train_df.drop('AverageCSAT', axis=1, inplace=True)

In [6]:
# boxcox transform label to see how normal its distribution and plot for comparison
y_train_arr,lmbd = stats.boxcox(y_train_df)

In [7]:
# label takes the Box Cox transform AverageCSAT
y_train_df = pd.DataFrame(y_train_arr)

## Build an XGBoost model for comparison

In [8]:
import xgboost as xgb

model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [9]:
from tpot import TPOTRegressor

ImportError: cannot import name 'LatentDirichletAllocation'

In [ ]:
model_xgb.fit(train_df,y_train_df)

In [ ]:
xgb_pred = model_xgb.predict(test_df)
# inverse boxcox transform
xgb_pred = inv_boxcox(xgb_pred, lmbd)

In [ ]:
# plot the difference between actual and XGB prediction score
fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(y_test_df.data,alpha=0.3)
ax.plot(xgb_pred.data)

ax.set_title('2019 CSAT and prediction')
ax.legend(['orig','prediction'],loc=3, prop={'size': 8})
ax.xaxis.set_label_text('cases')
ax.yaxis.set_label_text('CSAT Score')

plt.show();

In [ ]:
#len([(i,csat,xgb_pred[i]) for i,csat in enumerate(y_test_df) if (abs(y_test_df[i]-xgb_pred[i])>2.0)])
rms = np.sqrt(np.mean(np.square(y_test_df-xgb_pred)))
print(rms)

In [ ]:
# determine feature importance from trained model
feature_names = train_df.columns
coef = pd.Series(model_xgb.feature_importances_,feature_names).sort_values()

In [ ]:
coef

In [ ]:
# plot sorted feature importance
y_pos = np.arange(len(coef.index.array))
plt.figure(figsize=(6,4))
plt.bar(y_pos,coef.values,align='center', color='c', alpha=0.5)
plt.xticks(y_pos, coef.index.array)
plt.xticks(rotation='vertical')
plt.title("Case Metric Importance")
plt.show();

In [ ]:
from sklearn.feature_selection import SelectFromModel

thresholds = np.sort(model_xgb.feature_importances_)
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(model_xgb, threshold=thresh, prefit=True)
    select_X_train = selection.transform(train_df)
    print("{},{}".format(train_df.shape,select_X_train.shape))
    # train model
    selection_model = model_xgb
    selection_model.fit(select_X_train, y_train_df)
    # eval model
    select_X_test = selection.transform(test_df)
    y_pred = selection_model.predict(select_X_test)
    accuracy = np.sqrt(np.mean(np.square(y_test_df-y_pred)))
    print("Thresh=%.3f, n=%d, rms: %.2f%%" % (thresh, select_X_train.shape[1], accuracy))